# <center>YOLO Segmentation</center>

近年来，YOLO 系列在目标检测领域取得了显著成就，并进一步拓展到了目标分割领域，推出了 YOLO Segment 模型。这些模型不仅能够快速检测出图像中的目标，还能提供精确的分割掩码，帮助我们更好地理解目标的形状和边界。在本节课程中，我们将深入分析不同版本 YOLO 模型在分割任务中的异同，并通过可视化工具展示模型的输入输出结构，从而更好地理解其工作机制和性能表现。

## <center>YOLO Segmentation 结构分析</center>
<!-- 
&emsp;&emsp;Ultralytics OBB 模型的核心优势在于其能够处理旋转的边界框。与 Ultralytics Detect 模型相比，[Ultralytics OBB 模型的输出在每个边界框中多了一个表示旋转角度的参数](https://github.com/ultralytics/ultralytics/blob/v8.3.23/ultralytics/nn/modules/head.py#L217)。 -->

为了深入理解 YOLO Segmentation 模型的特性，我们首先需要对比不同版本 YOLO 模型在分割任务中的结构差异。以下是 YOLOv5、YOLOv7、YOLOv8、YOLOv9 和 YOLOv11 在分割任务中的结构分析，重点关注它们的输入输出格式以及分割掩码的生成方式。

In [ ]:
import netron
from IPython.display import IFrame, display

url, port = netron.start("./course_data/models/SEG/yolov5s-seg.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='520px'))

In [ ]:
url, port = netron.start("./course_data/models/SEG/yolov7-seg.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='520px'))

In [ ]:
url, port = netron.start("./course_data/models/SEG/yolov8n-seg.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='520px'))

In [ ]:
url, port = netron.start("./course_data/models/SEG/yolov9c-seg.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='520px'))

In [ ]:
url, port = netron.start("./course_data/models/SEG/yolo11n-seg.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='520px'))

与 YOLO Detection 模型类似，YOLO Segmentation 模型的输入节点同样接收一个包含 N 个样本的批次，每个样本是一张具有 3 个颜色通道、尺寸为高 × 宽像素的图像。不过，输出节点存在一些差异。YOLO Segmentation 模型**新增了一个维度为 `(batch, 32, w // 4, h // 4)` 的掩码原型图（Mask Prototypes）输出节点，并且在检测输出节点中额外增加了 32 个掩码系数，用于生成实例分割掩码。**

具体来说，通过将掩码系数与掩码原型图进行加权求和，即可生成目标的实例分割掩码。其计算公式如下：

$$
\text{mask} = \sum_{i=1}^{32}(\text{mask\_coefficients}_{i} \times \text{mask\_prototypes}_{i})
$$

在代码实现中，可以通过以下方式计算掩码：

```python
masks = torch.matmul(mask_coefficients, mask_prototypes)
```

## <center>Ultralytics YOLO Segment 注册自定义插件</center>

在 YOLO Detect 算法的变体中，通过集成 EfficientNMS 插件可以显著提高非极大值抑制（NMS）后处理的效率。然而，这个插件并不适用于 YOLO Segment 模型，因为 EfficientNMS 插件主要用于目标检测任务，其输出不包含实例分割所需的掩码系数，而实例分割需要通过掩码系数与掩码原型图结合来生成分割掩码。


为了解决这些问题，在之前的章节中我们开发了一个名为 EfficientIdxNMS 的自定义插件。相较于 EfficientNMS 插件，EfficientIdxNMS 多了一个 `detection_indices` 节点，用于获取筛选后的索引值，从而确保可以通过索引值获取到对应的掩码系数。这使得在实例分割任务中，能够高效地提取和处理分割掩码，进一步提升模型的推理效率。

In [1]:
import onnx
import onnxsim
import torch
from ultralytics import YOLO
from course_functions.head import UltralyticsSegment

def export_model(model_path: str, output_path: str, batch: int = 1, 
                 imgsz: tuple = (640, 640), max_boxes: int = 100, 
                 iou_thres: float = 0.45, conf_thres: float = 0.25, 
                 opset_version: int = 11) -> None:
    """
    将 YOLO 模型导出为 ONNX 格式，并修改检测头。
    
    参数：
    - model_path: YOLO 模型权重文件的路径。
    - output_path: 导出的 ONNX 模型保存路径。
    - batch: 模型的批量大小，默认为 1。
    - imgsz: 模型输入的图像尺寸，默认为 (640, 640)。
    - max_boxes: 每张图片的最大检测数量，默认为 100。
    - iou_thres: NMS 的 IoU 阈值，默认为 0.25。
    - conf_thres: 检测的置信度阈值，默认为 0.25。
    - opset_version: ONNX opset 版本，默认为 11。
    """
    # 加载指定权重的 YOLO 模型并设置为 CPU 模式
    model = YOLO(model=model_path, verbose=False).model.to('cpu')
    img_h, img_w = imgsz

    # 修改 Detect 层的参数
    for m in model.modules():
        if m.__class__.__name__ == "Segment":
            m.__class__ = type("Segment", (UltralyticsSegment,), {
                "dynamic": batch <= 0,  # 是否需要动态批量大小
                "max_det": max_boxes,  # 每张图片的最大检测数量
                "iou_thres": iou_thres,  # NMS 的 IoU 阈值
                "conf_thres": conf_thres,  # 检测的置信度阈值
            })
            break

    # 创建模型的虚拟输入张量
    dummy_input = torch.randn(1 if batch <= 0 else batch, 3, *imgsz).to('cpu')
    model.eval().float()  # 将模型设置为评估模式，并确保其为浮点精度
    # model(dummy_input)  # 使用虚拟输入运行模型，以确保其正确配置

    # 将模型导出为 ONNX 格式
    torch.onnx.export(
        model,
        dummy_input,
        output_path,
        export_params=True,
        opset_version=16,
        do_constant_folding=True,
        input_names=['images'],
        output_names=["num_dets", "det_boxes", "det_scores", "det_classes", "det_masks"],
        dynamic_axes={
            "images": {0: "batch", 2: "height", 3: "width"},
            "num_dets": {0: "batch"},
            "det_boxes": {0: "batch"},
            "det_scores": {0: "batch"},
            "det_classes": {0: "batch"},
            "det_masks": {0: "batch"},
        } if batch <= 0 else None,  # 如果批量大小是动态的，则定义动态轴
    )

    # 加载导出的 ONNX 模型并进行验证
    model_onnx = onnx.load(output_path)
    onnx.checker.check_model(model_onnx)

    # 则更新输出节点维度
    shapes = {
        'num_dets': ["batch" if batch <= 0 else batch, 1],
        'det_boxes': ["batch" if batch <= 0 else batch, max_boxes, 4],
        'det_scores': ["batch" if batch <= 0 else batch, max_boxes],
        'det_classes': ["batch" if batch <= 0 else batch, max_boxes],
        'det_masks': ["batch" if batch <= 0 else batch, max_boxes, "height" if batch <= 0 else img_h, "width" if batch <= 0 else img_w],
    }
    for node in model_onnx.graph.output:
        for idx, dim in enumerate(node.type.tensor_type.shape.dim):
            dim.dim_param = str(shapes[node.name][idx])

    # 简化 ONNX 模型
    model_onnx, check = onnxsim.simplify(model_onnx)
    assert check, "Simplified ONNX model could not be validated"
    onnx.save(model_onnx, output_path)  # 保存简化后的 ONNX 模型

# 调用函数导出模型
export_model("./course_data/models/SEG/yolo11n-seg.pt", "./course_data/models/SEG/yolo11n-seg_with_plugin.onnx")

/home/my_ubuntu/AI_deployment/TRT-TritonServer-Lesson/course_functions/head.py:287: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.shape != shape:
/home/my_ubuntu/myenv/lib/python3.10/site-packages/ultralytics/utils/tal.py:369: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for i, stride in enumerate(strides):
[W707 22:16:50.750842367 shape_type_inference.cpp:1995] Warning: The shape inference of TRT::EfficientIdxNMS_TRT type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateRelia

In [ ]:
url, port = netron.start("./course_data/models/SEG/yolo11n-seg_with_plugin.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='620px'))

<b><font color="red">作业：</font></b>对 [Ultralytics-YOLO Segment](https://github.com/ultralytics/ultralytics) 导出集成 EfficientIdxNMS 插件的 ONNX 模型后，相信大家已经摩拳擦掌，准备亲自动手实践了。那么，接下来的挑战是：尝试为[YOLOv5 Segment](https://github.com/ultralytics/yolov5) 导出集成 EfficientIdxNMS 插件的 ONNX 模型。

## <center>总结</center>

本节课程中通过对比 YOLOv5 至 YOLOv11 在分割任务中的结构差异，我们了解了各版本在输入输出格式和分割掩码生成方式上的特点。之后，我们介绍了如何为 YOLO Segmentation 模型注册自定义插件，特别是通过开发 EfficientIdxNMS 插件提升推理效率。